In [1]:
from collections import OrderedDict
import numpy as np
from qiskit import Aer
from qiskit.transpiler import PassManager

from qiskit_aqua import Operator, QuantumInstance
from qiskit_aqua.algorithms.adaptive import VQE
from qiskit_aqua.algorithms.classical import ExactEigensolver
from qiskit_aqua.algorithms.components.optimizers import L_BFGS_B
from qiskit_aqua.algorithms.components.variational_forms import RY

from qiskit_chemistry import FermionicOperator
from qiskit_chemistry.drivers import ConfigurationManager

In [2]:
# using driver to get fermionic Hamiltonian
# PyQuante example
cfg_mgr = ConfigurationManager()
pyquante_cfg = OrderedDict([('atoms', 'H .0 .0 .0; H .0 .0 0.735'), ('units', 'Angstrom'), ('charge', 0), ('multiplicity', 1), ('basis', 'sto3g')])
section = {}
section['properties'] = pyquante_cfg
driver = cfg_mgr.get_driver_instance('PYQUANTE')
molecule = driver.run(section)
h1 = molecule.one_body_integrals
h2 = molecule.two_body_integrals

In [3]:
# convert from fermionic hamiltonian to qubit hamiltonian
ferOp = FermionicOperator(h1=h1, h2=h2)
qubitOp_jw = ferOp.mapping(map_type='JORDAN_WIGNER', threshold=0.00000001)
qubitOp_pa = ferOp.mapping(map_type='PARITY', threshold=0.00000001)
qubitOp_bi = ferOp.mapping(map_type='BRAVYI_KITAEV', threshold=0.00000001)

In [4]:
# print out qubit hamiltonian in Pauli terms and exact solution

qubitOp_jw.to_matrix()
qubitOp_jw.chop(10**-10)

print(qubitOp_jw.print_operators())

# Using exact eigensolver to get the smallest eigenvalue
exact_eigensolver = ExactEigensolver(qubitOp_jw, k=1)
ret = exact_eigensolver.run()
print('The exact ground state energy is: {}'.format(ret['energy']))    

IIII	(-0.8105479862761009+0j)
IIIZ	(0.1721839427308564+0j)
IIZI	(-0.22575350251540538+0j)
IIZZ	(0.12091263358560006+0j)
IZII	(0.1721839427308564+0j)
IZIZ	(0.1689275404885901+0j)
IZZI	(0.16614543338049342+0j)
IZZZ	(-5.551115123125783e-17+0j)
XXXX	(0.045232799794893475+0j)
XXYY	(0.045232799794893475+0j)
YYXX	(0.045232799794893475+0j)
YYYY	(0.045232799794893475+0j)
ZIII	(-0.2257535025154054+0j)
ZIIZ	(0.16614543338049342+0j)
ZIZI	(0.17464343142442204+0j)
ZIZZ	(-5.551115123125783e-17+0j)
ZZII	(0.12091263358560007+0j)
ZZIZ	(-5.551115123125783e-17+0j)
ZZZI	(-4.163336342344337e-17+0j)
ZZZZ	(-2.7755575615628914e-17+0j)

The exact ground state energy is: -1.85727507663788


In [5]:
lbfgs = L_BFGS_B()
lbfgs.set_options(maxfun=1000, factr=10, iprint=10)

# setup variation form generator (generate trial circuits for VQE)
var_form = RY(qubitOp_jw.num_qubits, 5, entangler_map = {0: [1], 1:[2], 2:[3]})

# setup VQE with operator, variation form, and optimzer
vqe_algorithm = VQE(qubitOp_jw, var_form, lbfgs, 'matrix')

backend = Aer.get_backend('statevector_simulator')
quantum_instance = QuantumInstance(backend, pass_manager=PassManager())

results = vqe_algorithm.run(quantum_instance)

print("Minimum value: {}".format(results['eigvals'][0]))
print("Parameters: {}".format(results['opt_params']))

Minimum value: -1.857275076616748
Parameters: [ 1.24560170e-01 -2.24608226e+00 -3.14159048e+00 -2.42131530e+00
  8.72124461e-01 -2.06751067e+00 -6.50175294e-01  2.88665414e+00
  1.27536588e+00  4.63691897e-01 -1.92054319e+00  2.67567922e+00
 -2.34453122e-01  1.10136519e+00  1.69630611e+00  7.44171042e-01
  2.95271596e+00  2.30531994e-03  1.59835566e-01  7.98698092e-01
 -1.75134877e+00  9.96040972e-01  2.10338450e+00  2.03697896e+00]
